In [4]:
from src.utils.datasets import get_dataset
import torch
import numpy as np

import argparse
import random

import numpy as np
import torch

from src import config
from src.NICE_SLAM import NICE_SLAM
from src.Segmenter import Segmenter

import os  # J:added
from torch.utils.tensorboard import SummaryWriter  # J: added
import yaml  # J: added
from scripts import gifMaker  # J: added
import argparse

In [6]:
def get_bounds(margin, path_to_config):
    parser = argparse.ArgumentParser(
    description="Arguments for running the NICE-SLAM/iMAP*."
    )
    parser.add_argument("config", type=str, help="Path to config file.")
    parser.add_argument(
        "--input_folder",
        type=str,
        help="input folder, this have higher priority, can overwrite the one in config file",
    )
    parser.add_argument(
        "--output",
        type=str,
        help="output folder, this have higher priority, can overwrite the one in config file",
    )
    nice_parser = parser.add_mutually_exclusive_group(required=False)
    nice_parser.add_argument("--nice", dest="nice", action="store_true")
    nice_parser.add_argument("--imap", dest="nice", action="store_false")
    parser.set_defaults(nice=True)
    args = parser.parse_args(args=[f'/home/rozenberszki/project/wsnsl/configs/TUM_RGBD/{path_to_config}'])
    cfg = config.load_config(  # J:changed it to use our config file including semantics
            args.config, "configs/nice_slam_sem.yaml" if args.nice else "configs/imap.yaml"
        )
    slam = NICE_SLAM(cfg, args)
    frame_reader = get_dataset(cfg, args, cfg["scale"], slam = slam)
    camera_poses = frame_reader.poses
    camera_poses = torch.concat([c[None] for c in camera_poses]).float()
    translations = camera_poses[:, :3, 3]
    min_bounds = torch.min(translations, axis=0)[0] -margin
    max_bounds = torch.max(translations, axis=0)[0]+margin
    bounding_box = torch.concat([min_bounds[:,None], max_bounds[:,None]], axis=1)
    return bounding_box

In [3]:
get_bounds(2,'5fb5d2dbf2.yaml')

FileNotFoundError: [Errno 2] No such file or directory: '/home/rozenberszki/project/wsnsl/configs/TUM_RGBD/5fb5d2dbf2.yaml'

In [5]:
parser = argparse.ArgumentParser(
    description="Arguments for running the NICE-SLAM/iMAP*."
)
parser.add_argument("config", type=str, help="Path to config file.")
parser.add_argument(
    "--input_folder",
    type=str,
    help="input folder, this have higher priority, can overwrite the one in config file",
)
parser.add_argument(
    "--output",
    type=str,
    help="output folder, this have higher priority, can overwrite the one in config file",
)
nice_parser = parser.add_mutually_exclusive_group(required=False)
nice_parser.add_argument("--nice", dest="nice", action="store_true")
nice_parser.add_argument("--imap", dest="nice", action="store_false")
parser.set_defaults(nice=True)
args = parser.parse_args(args=['/home/rozenberszki/project/wsnsl/configs/TUM_RGBD/freiburg2_xyz.yaml'])
cfg = config.load_config(  # J:changed it to use our config file including semantics
        args.config, "configs/nice_slam_sem.yaml" if args.nice else "configs/imap.yaml"
    )
num_of_runs = (
    len(os.listdir(cfg["data"]["logs"]))
    if os.path.exists(cfg["data"]["logs"])
    else 0
)
path = os.path.join(cfg["data"]["logs"], f"run_{num_of_runs + 1}")
cfg["data"]["logs"] = path

slam = NICE_SLAM(cfg, args)

OSError: Datasets/TUM_RGBD/rgbd_dataset_freiburg2_xyz/rgb.txt not found.

In [9]:
frame_reader = get_dataset(cfg, args, cfg["scale"], slam = slam)

In [18]:
camera_poses = frame_reader.poses
camera_poses = torch.concat([c[None] for c in camera_poses]).float()
camera_poses.shape

torch.Size([3397, 4, 4])

In [20]:
translations = camera_poses[:, :3, 3]
translations.shape

torch.Size([3397, 3])

In [33]:

# Compute min and max for each dimension
min_bounds = torch.min(translations, axis=0)[0] -2
min_bounds

tensor([-2.5114, -2.3642, -3.0447])

In [34]:
max_bounds = torch.max(translations, axis=0)[0]+2
max_bounds

tensor([2.4333, 2.3098, 2.4352])

In [35]:
bounding_box = torch.concat([min_bounds[:,None], max_bounds[:,None]], axis=1)
bounding_box

tensor([[-2.5114,  2.4333],
        [-2.3642,  2.3098],
        [-3.0447,  2.4352]])